In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir("..")


In [ ]:
from src.data import Video
from src.distances import lcss_dist
from src.resources.joints import JOINTS
from src.visualization import plot_3d_landmarks


<img src="https://i.imgur.com/3j8BPdc.png" style="width:800px" >

In [ ]:
vid = Video(path=1)
vid.get_landmarks(show=True, model_complexity=1)
vid.extract_coordinates()

In [ ]:
vid.extract_angles(angle_list=JOINTS["left"])
vid._angles.plot()

In [ ]:
vid._angles.smooth(method="lowess", inplace=True)
vid._angles.plot()

In [ ]:
plot_3d_landmarks(
    landmark_list=a[-50],
    connections=mp.solutions.pose.POSE_CONNECTIONS,
)
